In [1]:
import pandas as pd
from datetime import datetime, date
from zoneinfo import ZoneInfo  # Python 3.9+

def build_date_dimension(
    start_date: str = "2020-01-01",
    end_date: str = "2030-12-31",
    fiscal_start_month: int = 7,          # 7 = July (AU)
    holidays: list[str] = None,
    anchor_today: date | str | None = None,
    tz_region: str = "Australia/Perth"    # <-- Pick your region here
) -> pd.DataFrame:
    # -------- base range --------
    rng = pd.date_range(start=start_date, end=end_date, freq="D")
    df = pd.DataFrame({"Date": rng})

    # -------- anchor (for offsets) --------
    if anchor_today is None:
        local_today_date = datetime.now(ZoneInfo(tz_region)).date()
    else:
        local_today_date = pd.to_datetime(anchor_today).date()
    today = pd.Timestamp(local_today_date)

    # -------- calendar attributes --------
    df["DateKey"] = df["Date"].dt.strftime("%Y%m%d").astype(int)

    df["Day"] = df["Date"].dt.day
    df["DayOfWeek"] = df["Date"].dt.weekday  # Monday=0
    df["DayName"] = df["Date"].dt.day_name()

    iso = df["Date"].dt.isocalendar()
    df["ISOWeek"] = iso.week.astype(int)   # Monday–Sunday week numbering
    df["ISOYear"] = iso.year.astype(int)

    df["Month"] = df["Date"].dt.month
    df["MonthName"] = df["Date"].dt.month_name()
    df["MonthShortName"] = df["Date"].dt.strftime("%b")
    df["Quarter"] = df["Date"].dt.quarter
    df["Year"] = df["Date"].dt.year
    df["IsLeapYear"] = df["Date"].dt.is_leap_year

    # Quarter labels (calendar)
    df["QuarterName"] = "Q" + df["Quarter"].astype(str)  # Q1..Q4
    df["QuarterYearShort"] = df["QuarterName"] + " " + (df["Year"] % 100).astype(str).str.zfill(2)  # e.g., Q1 25

    # Starts/ends (calendar)
    df["StartOfWeek"] = df["Date"] - pd.to_timedelta(df["Date"].dt.weekday, unit="D")
    df["EndOfWeek"] = df["StartOfWeek"] + pd.Timedelta(days=6)

    per_m = df["Date"].dt.to_period("M")
    df["StartOfMonth"] = per_m.dt.start_time
    df["EndOfMonth"] = per_m.dt.end_time

    per_q = df["Date"].dt.to_period("Q")  # Q-DEC
    df["StartOfQuarter"] = per_q.dt.start_time
    df["EndOfQuarter"] = per_q.dt.end_time

    per_y = df["Date"].dt.to_period("A-DEC")
    df["StartOfYear"] = per_y.dt.start_time
    df["EndOfYear"] = per_y.dt.end_time

    # -------- fiscal attributes (FY end month = fiscal_start_month-1) --------
    end_month_map = {1:"A-JAN",2:"A-FEB",3:"A-MAR",4:"A-APR",5:"A-MAY",6:"A-JUN",
                     7:"A-JUL",8:"A-AUG",9:"A-SEP",10:"A-OCT",11:"A-NOV",12:"A-DEC"}
    a_code = end_month_map[((fiscal_start_month + 10) % 12) + 1]  # month before start
    q_code = "Q-" + a_code.split("-")[1]

    fy_period = df["Date"].dt.to_period(a_code)     # e.g., A-JUN (AU)
    fq_period = df["Date"].dt.to_period(q_code)     # e.g., Q-JUN

    df["FiscalYear"] = fy_period.dt.year            # FY labeled by end year
    df["StartOfFiscalYear"] = fy_period.dt.start_time
    df["EndOfFiscalYear"] = fy_period.dt.end_time

    df["FiscalQuarter"] = fq_period.dt.quarter
    df["StartOfFiscalQuarter"] = fq_period.dt.start_time
    df["EndOfFiscalQuarter"] = fq_period.dt.end_time

    df["FiscalMonth"] = ((df["Month"] - fiscal_start_month) % 12) + 1

    # FY labels
    fy_end = df["FiscalYear"]
    fy_start = fy_end - 1
    df["FiscalYearLabel"] = "FY" + fy_end.astype(str)
    df["FiscalYearLabelShort"] = (
        "FY" + (fy_start % 100).astype(str).str.zfill(2) + "/" + (fy_end % 100).astype(str).str.zfill(2)
    )

    df["FiscalQuarterName"] = "Q" + df["FiscalQuarter"].astype(str)
    df["FiscalQuarterYearShort"] = df["FiscalQuarterName"] + " " + (df["FiscalYear"] % 100).astype(str).str.zfill(2)
    df["FiscalQuarterLabel"] = df["FiscalQuarterName"] + " " + df["FiscalYearLabel"]
    df["FiscalMonthName"] = df["Date"].dt.month_name()
    df["FiscalMonthLabel"] = df["FiscalMonthName"] + " " + df["FiscalYearLabel"]

    # Week of fiscal year (1..53) measured from StartOfFiscalYear
    df["WeekOfFiscalYear"] = ((df["Date"] - df["StartOfFiscalYear"]).dt.days // 7) + 1

    # Explicit fiscal month starts/ends (same as calendar month boundaries)
    df["StartOfFiscalMonth"] = df["StartOfMonth"]
    df["EndOfFiscalMonth"] = df["EndOfMonth"]

    # -------- weeks (labels) --------
    df["WeekOfYear"] = df["ISOWeek"]                 # 1..53 (no padding)
    df["WeekLabel"] = "Week " + df["WeekOfYear"].astype(str)      # "Week 1"
    df["WeekLabelShort"] = "Wk " + df["WeekOfYear"].astype(str)   # "Wk 1"

    # -------- offsets (relative to 'today') --------
    today_week_start = today - pd.to_timedelta(today.weekday(), unit="D")
    today_year = today.year
    today_month = today.month
    today_quarter = pd.Timestamp(today).quarter

    df["Offset_Day"] = (df["Date"] - today).dt.days
    df["Offset_Week"] = ((df["StartOfWeek"] - today_week_start).dt.days // 7)  # week offset (Mon–Sun)
    df["Offset_Month"] = ((df["Year"] - today_year) * 12 + (df["Month"] - today_month))
    df["Offset_Quarter"] = ((df["Year"] * 4 + df["Quarter"]) - (today_year * 4 + today_quarter))
    df["Offset_Year"] = df["Year"] - today_year

    # Fiscal offsets (FY-aligned)
    fy_today = pd.Period(today, freq=a_code).year
    fq_today = pd.Period(today, freq=q_code).quarter
    fmonth_today = ((today_month - fiscal_start_month) % 12) + 1

    df["Offset_FiscalYear"] = df["FiscalYear"] - fy_today
    df["Offset_FiscalQuarter"] = (df["FiscalYear"] * 4 + df["FiscalQuarter"]) - (fy_today * 4 + fq_today)
    df["Offset_FiscalMonth"] = (df["FiscalYear"] * 12 + df["FiscalMonth"]) - (fy_today * 12 + fmonth_today)

    # -------- NEW: human-friendly labels + sort fields (add-if-missing) --------
    def add_if_missing(col_name: str, series: pd.Series):
        if col_name not in df.columns:
            df[col_name] = series

    # Month-Year labels
    add_if_missing("MonthYear",       df["MonthName"] + " " + df["Year"].astype(str))          # March 2025
    add_if_missing("MonthYearShort",  df["Date"].dt.strftime("%b %y"))                         # Mar 25
    # Sort for month-year (also use for YearQuarterMonth*)
    add_if_missing("MonthYearSort",   df["Year"] * 100 + df["Month"])

    # Year-Quarter labels
    add_if_missing("YearQuarter",     df["Year"].astype(str) + " " + df["QuarterName"])        # 2025 Q1
    add_if_missing("YearQuarterSort", df["Year"] * 10 + df["Quarter"])                         # sort by Y then Q

    # Year-Quarter-Month labels (long + short)
    add_if_missing("YearQuarterMonth",      df["YearQuarter"] + " " + df["MonthName"])         # 2025 Q1 March
    add_if_missing("YearQuarterMonthShort", (df["Year"] % 100).astype(str).str.zfill(2)
                                               + " " + df["QuarterName"] + " "
                                               + df["Date"].dt.strftime("%b"))                 # 25 Q1 Mar
    # Sorting for Y-Q-M: reuses month-year sort (Y then M is sufficient & intuitive)
    add_if_missing("YearQuarterMonthSort",  df["MonthYearSort"])

    # -------- convenience flags --------
    df["IsToday"] = df["Date"].dt.date == today.date()
    df["IsWeekend"] = df["Date"].dt.weekday >= 5

    # Holidays (optional)
    if holidays:
        hol = pd.to_datetime(holidays).normalize()
        df["IsHoliday"] = df["Date"].isin(hol)
    else:
        df["IsHoliday"] = False

    # -------- final typing: convert boundary timestamps to date for lakehouse friendliness --------
    date_cols = [
        "Date",
        "StartOfWeek","EndOfWeek",
        "StartOfMonth","EndOfMonth",
        "StartOfQuarter","EndOfQuarter",
        "StartOfYear","EndOfYear",
        "StartOfFiscalMonth","EndOfFiscalMonth",
        "StartOfFiscalQuarter","EndOfFiscalQuarter",
        "StartOfFiscalYear","EndOfFiscalYear",
    ]
    for c in date_cols:
        df[c] = pd.to_datetime(df[c]).dt.date

    return df


StatementMeta(, 2c2bda0f-ec1e-483f-a345-d6668789efbb, 3, Finished, Available, Finished)

In [2]:

# Generate the dimension in pandas (Silver build)
df_date = build_date_dimension(
    start_date="2000-01-01",
    end_date="2040-12-31",
    fiscal_start_month=7,  # AU FY = July
    holidays=["2024-01-01","2024-03-29","2024-04-01","2024-04-25","2024-12-25"] , # optional
    tz_region="Australia/Perth"  # ensures Offset_* uses AWST dates
)

# Convert pandas -> Spark
sdf_date = spark.createDataFrame(df_date)


# Write as a managed Delta table (idempotent overwrite)
(
    sdf_date.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable("Silver.date_dimension")
)




StatementMeta(, 2c2bda0f-ec1e-483f-a345-d6668789efbb, 4, Finished, Available, Finished)